In [1]:
import pandas_datareader as pdr

In [3]:
df = pdr.get_data_tiingo('AAPL', api_key='ab2882f6485e2828c46882b863cc7c8cdaaef007')
df.head()

C:\Users\sinch\AppData\Local\Temp\ipykernel_13280\918027198.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df = pdr.get_data_tiingo('AAPL', api_key='ab2882f6485e2828c46882b863cc7c8cdaaef007')


close    high      low    open    volume  \
symbol date                                                                   
AAPL   2017-12-07 00:00:00+00:00  169.32  170.44  168.910  169.03  24469613   
       2017-12-08 00:00:00+00:00  169.37  171.00  168.820  170.49  23096872   
       2017-12-11 00:00:00+00:00  172.67  172.89  168.790  169.20  33092051   
       2017-12-12 00:00:00+00:00  171.70  172.39  171.461  172.15  18945457   
       2017-12-13 00:00:00+00:00  172.27  173.54  172.000  172.50  23142242   

                                   adjClose    adjHigh     adjLow    adjOpen  \
symbol date                                                                    
AAPL   2017-12-07 00:00:00+00:00  40.249729  40.515968  40.152266  40.180792   
       2017-12-08 00:00:00+00:00  40.261615  40.649088  40.130872  40.527854   
       2017-12-11 00:00:00+00:00  41.046071  41.098368  40.123740  40.221203   
       2017-12-12 00:00:00+00:00  40.815488  40.979511  40.758674  40.922459   
       2017-12-13 00:00:00+00:00  40.950985  41.252882  40.886802  41.005659   

                                  adjVolume  divCash  splitFactor  
symbol date                                                        
AAPL   2017-12-07 00:00:00+00:00   97878452      0.0          1.0  
       2017-12-08 00:00:00+00:00   92387488      0.0          1.0  
       2017-12-11 00:00:00+00:00  132368204      0.0          1.0  
       2017-12-12 00:00:00+00:00   75781828      0.0          1.0  
       2017-12-13 00:00:00+00:00   92568968      0.0          1.0

In [4]:
df2 = df.reset_index()['close']
df2.head()

0    169.32
1    169.37
2    172.67
3    171.70
4    172.27
Name: close, dtype: float64